In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
air_pollution <- read_csv("PRSA_Data_Aotizhongxin_20130301-20170228.csv")

Rows: 35064 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): wd, station
dbl (16): No, year, month, day, hour, PM2.5, PM10, SO2, NO2, CO, O3, TEMP, P...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
install.packages("con2aqi")
library(con2aqi)
air_poll_nona <- na.omit(air_pollution)
air_poll_new <- air_poll_nona |>
    mutate(SO2 = SO2/2.62, NO2 = NO2/1.88, O3 = O3/1960, CO = CO/1150)|>
    filter(PM2.5 <= 500.4, PM10 <= 604, SO2 <= 1004, NO2 <= 2049, CO <= 50.4, O3 <= 0.604)
pm25_aqi <- con2aqi(pollutant = "pm25", con = air_poll_new$PM2.5)|>
    as.data.frame()
colnames(pm25_aqi) <- "pm25_aqi"
pm10_aqi <- con2aqi(pollutant = "pm10", con = air_poll_new$PM10)|>
    as.data.frame()
colnames(pm10_aqi) <- "pm10_aqi"
so2_aqi <- con2aqi(pollutant = "so2", con = air_poll_new$SO2)|>
    as.data.frame()
colnames(so2_aqi) <- "so2_aqi"
no2_aqi <- con2aqi(pollutant = "no2", con = air_poll_new$NO2)|>
    as.data.frame()
colnames(no2_aqi) <- "no2_aqi"
co_aqi <- con2aqi(pollutant = "co", con = air_poll_new$CO)|>
    as.data.frame()
colnames(co_aqi) <- "co_aqi"
o3_aqi <- con2aqi(pollutant = "o3", con = air_poll_new$O3, type = "1h")|>
    as.data.frame()
colnames(o3_aqi) <- "o3_aqi"

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
air_poll_aqi <- bind_cols(air_poll_new, pm25_aqi, pm10_aqi, so2_aqi, no2_aqi, co_aqi, o3_aqi)
air_poll_aqi <- air_poll_aqi |>
    select(-wd, -station)|>
    group_by(year, month, day) |>
    summarize(across(TEMP:o3_aqi, mean))|>
    rowwise()|>
    mutate(aqi = round(max(pm25_aqi:o3_aqi)))
air_poll_aqi

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


year,month,day,TEMP,PRES,DEWP,RAIN,WSPM,pm25_aqi,pm10_aqi,so2_aqi,no2_aqi,co_aqi,o3_aqi,aqi
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,3,1,1.3916667,1026.875,-18.7458333,0.000000000,3.254167,28.91667,10.37500,6.750000,12.04167,4.291667,64.45833,64
2013,3,2,0.6166667,1026.850,-15.9375000,0.000000000,1.479167,81.95833,34.66667,20.458333,34.20833,8.250000,57.45833,82
2013,3,3,5.5666667,1014.608,-12.3166667,0.000000000,1.658333,151.29167,84.08333,33.500000,41.29167,16.208333,55.37500,151
2013,3,4,10.3714286,1017.714,-11.6857143,0.000000000,2.361905,74.09524,38.66667,13.619048,25.90476,6.428571,62.09524,74
2013,3,5,6.2916667,1010.900,-7.5250000,0.000000000,1.129167,209.54167,115.62500,51.000000,69.12500,23.583333,65.04167,210
2013,3,6,7.3260870,1007.491,-4.1347826,0.000000000,1.486957,275.26087,160.56522,62.652174,74.52174,31.086957,55.52174,275
2013,3,7,9.2875000,1002.321,-2.0333333,0.000000000,1.558333,313.33333,190.12500,52.416667,75.33333,36.250000,59.29167,313
2013,3,8,10.3000000,999.595,-1.0850000,0.000000000,1.780000,262.65000,163.20000,36.100000,58.20000,24.800000,66.95000,263
2013,3,9,6.4000000,1004.119,-9.3190476,0.000000000,2.838095,123.00000,119.04762,20.666667,25.95238,13.047619,66.28571,123
